#### Building a ChatBot with history

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002188AB3E260>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002188AB5C7F0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Mudith. I'm a Bioinformatician")])

AIMessage(content="Hi Mudith, it's nice to meet you! \n\nAs a large language model, I'm always eager to learn more about different fields.  Bioinformatics is a fascinating area.  What kind of work do you do in bioinformatics?  Are you working on any exciting projects right now?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 24, 'total_tokens': 90, 'completion_time': 0.12, 'prompt_time': 0.00015159, 'queue_time': 0.013318379, 'total_time': 0.12015159}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-46923f31-8e47-4bbb-a40c-b5a4154b5fbb-0', usage_metadata={'input_tokens': 24, 'output_tokens': 66, 'total_tokens': 90})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi my name is Mudith. I'm a Bioinformatician"),
        AIMessage(content="Hi Mudith, \n\nIt's nice to meet you! \n\nWhat kind of bioinformatic work do you do?  I'm always interested in learning more about the fascinating field of bioinformatics. Do you have any particular areas of expertise or projects you're working on that you'd like to talk about?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Mudith, a Bioinformatician.  \n\nIs there anything else you'd like to know or discuss? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 113, 'total_tokens': 143, 'completion_time': 0.054545455, 'prompt_time': 0.003862399, 'queue_time': 0.010009520000000001, 'total_time': 0.058407854}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed818c56-e6cd-404c-9a71-9df038b19673-0', usage_metadata={'input_tokens': 113, 'output_tokens': 30, 'total_tokens': 143})

##### Message History

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Muditha and I'm a Bioinformatician")],
    config=config
)

In [10]:
response.content

"Hello Muditha!\n\nIt's nice to meet you.  \n\nIt seems like you're passionate about bioinformatics. What kind of work do you do in this field?  I'm always interested in learning more about how people use data to understand biological systems. \n\n"

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config,
)

AIMessage(content='Your name is Muditha. 😊  \n\nI remember from our earlier conversation.  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 99, 'total_tokens': 120, 'completion_time': 0.038181818, 'prompt_time': 0.002630102, 'queue_time': 0.011583837999999999, 'total_time': 0.04081192}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-417844f9-4577-4e4c-b1a4-01d6bf39cf8e-0', usage_metadata={'input_tokens': 99, 'output_tokens': 21, 'total_tokens': 120})

In [12]:
## changing the config----> session id
config1={"configurable":{"session_id":"chat2"}}

response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey my name is John")],
    config=config1
)

response.content

"Hey John! It's nice to meet you.  \n\nIs there anything I can help you with today?\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)

response.content

'Your name is John, is that right?  \n\nYou told me earlier!  😄  \n'

#### Prompt templates

In [29]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
    ("system","You are a helpful assistance. Anser all the questions to the best of your knowledge"),
    MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [30]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Mudith")]})
response.content

"Hi Mudith! It's nice to meet you. \n\nHow can I help you today? 😊  \n\n"

In [31]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [32]:
config={"configurable":{"session_id":"chat3"}}

response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is Mudith")],
    config=config,
)

response.content

"Hi Mudith, it's nice to meet you! \n\nWhat can I do for you today? 😊  \n"

In [33]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config,
)

response.content

'Your name is Mudith.  \n\nI remember!  😊  \n'

In [34]:
## Adding more complexity

prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system","You are a helpful assistance. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [25]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Mudith")],"language":"Sinhala"})
response.content

'අපෙන් ආයුබෝවන්! මුදිත් නම් ඔබේ නම ඉතා සුන්දරයි. \n\nමට සැලකෙන ආකාරයෙන් මම ඔබේ උපකාර ලබා දීමට සූදානම් වෙමි. \n\nඔබට කිව්වේද? \n'

In [35]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [36]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="What is my name?")],"language":"Sinhala"},
    config=config,
)

response.content

'මේ වර්තමානයේ ඔබේ නම Mudith. 😊\n\nAre you curious about your name? \n'

#### Managing the conversation History

In [37]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

C:\Users\Dhananjana\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Dhananjana\Desktop\LLM\llm_implementation\llm\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dhananjana\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In ord

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [50]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have taste buds or personal preferences like favorite ice cream flavors!  \n\nWhat's your favorite flavor?  🍦😊  \n\n\n\n"

In [51]:
response=chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did I ask?")],
        "language":"English",
    }
)

response.content

'You asked  "what is 2 + 2?" \n'

In [52]:
## Wrapping this with message history

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config={"configurable":{"session_id":"chat5"}}

In [53]:
response=with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my name?")],
        "language":"English",
    },
    config=config
)

response.content

"As an AI, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [54]:
response=with_message_history.invoke(
    {
        "messages":[HumanMessage(content="What math problem did I ask?")],
        "language":"English",
    },
    config=config
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n"